In [1]:
# crear puntos de entrada para spark
try:
    sc.stop()
except:
    pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark = SparkSession(sparkContext=sc)

## Funciones Map

Estas funciones son probablemente las más utilizadas cuando se trata de un objeto RDD.

* `map()`
* `mapValues()`
* `flatMap()`
* `flatMapValues()`

### `map`

El método `map()` aplica una función a cada elemento del RDD. Cada elemento tiene que ser una entrada válida a la función. El RDD devuelto tiene las salidas de función como nuevos elementos.

Los elementos en el objeto RDD `map_exp_rdd` a continuación son filas de los[mtcars](data/mtcars.csv) en formato de cadena. Vamos a aplicar la función `map()` varias veces para convertir cada elemento de la cadena como un elemento de la lista. Cada elemento de la lista tiene dos valores: el primer valor será el modelo automático en formato de cadena; el segundo valor será una lista de valores numéricos.

In [2]:
# crear RDD ejemplo
map_exp_rdd = sc.textFile('../../data/mtcars.csv')
map_exp_rdd.take(4)

[',mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb',
 'Mazda RX4,21,6,160,110,3.9,2.62,16.46,0,1,4,4',
 'Mazda RX4 Wag,21,6,160,110,3.9,2.875,17.02,0,1,4,4',
 'Datsun 710,22.8,4,108,93,3.85,2.32,18.61,1,1,4,1']

In [3]:
# dividir el modelo automático de otras caracteristicas de valores
map_exp_rdd_1 = map_exp_rdd.map(lambda x: x.split(',')).map(lambda x: (x[0], x[1:]))
map_exp_rdd_1.take(4)

[('',
  ['mpg',
   'cyl',
   'disp',
   'hp',
   'drat',
   'wt',
   'qsec',
   'vs',
   'am',
   'gear',
   'carb']),
 ('Mazda RX4',
  ['21', '6', '160', '110', '3.9', '2.62', '16.46', '0', '1', '4', '4']),
 ('Mazda RX4 Wag',
  ['21', '6', '160', '110', '3.9', '2.875', '17.02', '0', '1', '4', '4']),
 ('Datsun 710',
  ['22.8', '4', '108', '93', '3.85', '2.32', '18.61', '1', '1', '4', '1'])]

In [4]:
# remover el header row
header = map_exp_rdd_1.first()
# el método de filtro aplica una función a cada elemento. La salida de la función es un valor booleano (TRUE or FALSE)
# que tienen la salida TRUE se mantendrán.
map_exp_rdd_2 = map_exp_rdd_1.filter(lambda x: x != header)
map_exp_rdd_2.take(4)

[('Mazda RX4',
  ['21', '6', '160', '110', '3.9', '2.62', '16.46', '0', '1', '4', '4']),
 ('Mazda RX4 Wag',
  ['21', '6', '160', '110', '3.9', '2.875', '17.02', '0', '1', '4', '4']),
 ('Datsun 710',
  ['22.8', '4', '108', '93', '3.85', '2.32', '18.61', '1', '1', '4', '1']),
 ('Hornet 4 Drive',
  ['21.4', '6', '258', '110', '3.08', '3.215', '19.44', '1', '0', '3', '1'])]

In [5]:
# convertir valores de cadena en valores numéricos
map_exp_rdd_3 = map_exp_rdd_2.map(lambda x: (x[0], list(map(float, x[1]))))
map_exp_rdd_3.take(4)

[('Mazda RX4',
  [21.0, 6.0, 160.0, 110.0, 3.9, 2.62, 16.46, 0.0, 1.0, 4.0, 4.0]),
 ('Mazda RX4 Wag',
  [21.0, 6.0, 160.0, 110.0, 3.9, 2.875, 17.02, 0.0, 1.0, 4.0, 4.0]),
 ('Datsun 710',
  [22.8, 4.0, 108.0, 93.0, 3.85, 2.32, 18.61, 1.0, 1.0, 4.0, 1.0]),
 ('Hornet 4 Drive',
  [21.4, 6.0, 258.0, 110.0, 3.08, 3.215, 19.44, 1.0, 0.0, 3.0, 1.0])]

### `mapValues`

Los `mapValues` requiere que cada elemento en el RDD tenga un valor de **key/value** estructura de pares, por ejemplo, una tupla de 2 posiciones o una lista de 2 posiciones. La función `mapValues` aplica una función a cada uno de los valores de los elementos. La clave del elemento permanecerá inalterada.

Podemos aplicar la función `mapValues` al objeto RDD `mapValues_exp_rdd` a continuación.


In [6]:
mapValues_exp_rdd = map_exp_rdd_3
mapValues_exp_rdd.take(4)

[('Mazda RX4',
  [21.0, 6.0, 160.0, 110.0, 3.9, 2.62, 16.46, 0.0, 1.0, 4.0, 4.0]),
 ('Mazda RX4 Wag',
  [21.0, 6.0, 160.0, 110.0, 3.9, 2.875, 17.02, 0.0, 1.0, 4.0, 4.0]),
 ('Datsun 710',
  [22.8, 4.0, 108.0, 93.0, 3.85, 2.32, 18.61, 1.0, 1.0, 4.0, 1.0]),
 ('Hornet 4 Drive',
  [21.4, 6.0, 258.0, 110.0, 3.08, 3.215, 19.44, 1.0, 0.0, 3.0, 1.0])]

In [7]:
import numpy as np
mapValues_exp_rdd_1 = mapValues_exp_rdd.mapValues(lambda x: np.mean(x))
mapValues_exp_rdd_1.take(4)

[('Mazda RX4', 29.90727272727273),
 ('Mazda RX4 Wag', 29.98136363636364),
 ('Datsun 710', 23.59818181818182),
 ('Hornet 4 Drive', 38.73954545454546)]

Cuando se utiliza mapValues(), la x en la función lambda anterior se refiere al valor del elemento, sin incluir la clave del elemento.

### `flatMap`

Esta función aplica primero una función a cada uno de los elementos de un RDD y luego aplana los resultados. Simplemente podemos utilizar esta función para aplanar elementos de un RDD sin necesidad de realizar ninguna operación adicional en cada uno de ellos.


In [8]:
x = [('a', 'b', 'c'), ('a', 'a'), ('c', 'c', 'c', 'd')]
flatMap_exp_rdd = sc.parallelize(x)
flatMap_exp_rdd.collect()

[('a', 'b', 'c'), ('a', 'a'), ('c', 'c', 'c', 'd')]

In [9]:
flatMap_exp_rdd_1 = flatMap_exp_rdd.flatMap(lambda x: x)
flatMap_exp_rdd_1.collect()

['a', 'b', 'c', 'a', 'a', 'c', 'c', 'c', 'd']

### `flatMapValues`

Los `flatMapValues` requiere que cada elemento en el RDD tenga una estructura de pares **clave/valor**. Aplica una función a cada **valor de elemento** del objeto RDD y luego aplana los resultados.

Por ejemplo, mis datos en bruto se ven como se muestra a continuación. Pero me gustaría transformar los datos para que tengan tres columnas: la primera columna es el **id de la muestra**; la segunda columna son los tres **tipos (A, B o C)**; la tercera columna son los **valores**.

| sample id |  A |  B |  C |
|:---------:|:--:|:--:|:--:|
|     1     | 23 | 18 | 32 |
|     2     | 18 | 29 | 31 |
|     3     | 34 | 21 | 18 |

In [10]:
# datos ejemplo
my_data = [
    [1, (23, 28, 32)],
    [2, (18, 29, 31)],
    [3, (34, 21, 18)]
]
flatMapValues_exp_rdd = sc.parallelize(my_data)
flatMapValues_exp_rdd.collect()

[[1, (23, 28, 32)], [2, (18, 29, 31)], [3, (34, 21, 18)]]

In [11]:
# fusionar las columnas A, B y C en una columna y añadir la columna de tipo
flatMapValues_exp_rdd_1 = flatMapValues_exp_rdd.flatMapValues(lambda x: list(zip(list('ABC'), x)))
flatMapValues_exp_rdd_1.collect()

[(1, ('A', 23)),
 (1, ('B', 28)),
 (1, ('C', 32)),
 (2, ('A', 18)),
 (2, ('B', 29)),
 (2, ('C', 31)),
 (3, ('A', 34)),
 (3, ('B', 21)),
 (3, ('C', 18))]

In [12]:
# desempaquetar los valores de los elementos
flatMapValues_exp_rdd_2 = flatMapValues_exp_rdd_1.map(lambda x: [x[0]] + list(x[1]) )
flatMapValues_exp_rdd_2.collect()

[[1, 'A', 23],
 [1, 'B', 28],
 [1, 'C', 32],
 [2, 'A', 18],
 [2, 'B', 29],
 [2, 'C', 31],
 [3, 'A', 34],
 [3, 'B', 21],
 [3, 'C', 18]]